In [1]:
import pandas as pd
from collections import defaultdict
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline  
%autosave 300
from xgboost import XGBClassifier

Autosaving every 300 seconds


In [2]:
pd.options.display.max_columns = None
feature_names =defaultdict(lambda:'None')

In [3]:
city_indicators=pd.read_csv('data/City Indicators.csv')
drought_data=pd.read_csv('data/Drought Data.csv')
flood_data=pd.read_csv('data/Flood Data.csv')
heat_data=pd.read_csv('data/Heat Data.csv')
historical=pd.read_csv('data/Historical Hazards Outcome Data.csv')
risk_recovery=pd.read_csv('data/Overall Risk & Readiness Scores.csv')
sea_level_data=pd.read_csv('data/Sea Level Rise Data.csv')



In [4]:
df=city_indicators.copy()
df.shape
#Counter(df['Historical Average Cost of Cold Event'])


(278, 53)

In [5]:
df.corr()

,Percent of GDP based on water intensive industries_2015,Percentage of workforce in Farming Fishing and Forestry 2015,Baseline water stress_2014,Readiness to accept adaptation investment_2015,Tax incentives for renewable energy_2015,Total number of federal public corruption convictions by district_2015,Civic engagement_2010,General innovation capabilities_2015,Estimated percent of adults who think global warming is already harming people in the US now or within 10 years_2014,Percent of population with less than 12th grade education_2015,Population density_2015,Water quality_2015,City debt_2010,Percent of population in floodzone_2015,Percent of buildings in floodzone_2015,Percent of cars in floodzone_2015,Percent of area that is impervious surface_2011,Number of acute care hospital beds available per 1000 residents_2015,Percent of population that is 65 years old or older living alone_2015,Percent of population that is 5 years old or younger_2015,Percent of population with health insurance_2015,Percent of buildings built before 1999_2015,Percent of population residing in the mobile homes_2015,Percent of population spending over 50 percent of income on rent_2015,Percent of land covered by tree canopy_2011,Percent of houses with heating_2015,Percent of population with a disability_2015,Percent of population in poverty_2015,Percent of buildings built before 1979_2015,Outdoor workers_2015,Percent of households without access to a vehicle_2015,Percent of population in 1ft sea level rise zone_2015,Percent of population in 3ft sea level rise zone_2015,Average economic impact of 1ft sea level rise_2015,Average economic impact of 3ft sea level rise_2015,Unnamed: 40,Latitutde,Longitude,City Area_KM,2010 Population,2011 Population,2012 Population,2013 Population,2014 Population,2015 Population,Median Income
Percent of GDP based on water intensive industries_2015,1.000000,0.107647,-0.102995,0.000013,-0.169631,-0.125156,0.029982,0.121448,-0.025751,0.036527,-0.274790,0.022715,-0.089990,0.010829,0.011327,0.062850,-0.041980,-0.027052,-0.001855,0.197543,-0.059035,0.033192,0.089106,-0.070622,-0.018752,0.164833,0.021329,-0.027463,0.007553,0.135317,-0.222953,-0.051135,-0.071992,0.000526,-0.075092,NaN,0.081618,-0.171406,0.023771,-0.077187,-0.076802,-0.076716,-0.076497,-0.076334,-0.075819,0.022869
Percentage of workforce in Farming Fishing and Forestry 2015,0.107647,1.000000,0.106905,-0.061655,-0.012007,-0.059903,-0.096002,0.064658,-0.035118,0.034678,-0.067619,-0.001775,-0.065818,-0.045049,-0.043982,0.012218,-0.051103,-0.006427,0.010355,0.082539,-0.040084,-0.003083,0.072871,0.016801,-0.029381,0.014939,-0.013428,-0.021956,-0.027696,0.062913,-0.048830,-0.106259,-0.068923,0.063124,0.000184,NaN,-0.016227,-0.072015,-0.012287,-0.029525,-0.029450,-0.029446,-0.029510,-0.029599,-0.029705,-0.029036
Baseline water stress_2014,-0.102995,0.106905,1.000000,-0.099617,0.139674,0.117970,-0.020957,0.163022,-0.082691,0.054763,0.069629,0.123275,-0.087724,-0.007681,-0.008673,-0.141618,0.045682,-0.105864,-0.119211,0.031102,-0.014933,-0.003394,0.069762,-0.106522,-0.226750,-0.120526,-0.166028,-0.174383,-0.028698,-0.014814,-0.082347,-0.066486,-0.106845,-0.218173,-0.198107,NaN,-0.081981,-0.108500,-0.041796,-0.054532,-0.053961,-0.053539,-0.053203,-0.052944,-0.052716,0.176042
Readiness to accept adaptation investment_2015,0.000013,-0.061655,-0.099617,1.000000,0.079734,-0.251227,0.120091,-0.059081,0.064194,-0.144306,-0.024840,0.123943,0.239560,0.070766,0.064848,0.033319,-0.018542,0.186065,0.196585,0.007251,0.046336,0.075244,-0.231737,-0.174847,0.322072,0.148361,0.106926,0.170415,0.140387,-0.129797,0.279848,0.102064,0.163571,0.055796,0.129088,NaN,0.209808,0.323706,0.170775,0.146628,0.146867,0.146978,0.147381,0.147677,0.148245,-0.186755
Tax incentives for renewable energy_2015,-0.169631,-0.012007,0.139674,0.079734,1.000000,-0.085831,0.001394,0.062818,-0.337299,0.051018,0.297657,0.123474,0.216038,0.013732,0.016585,-0.032664,-0.015128,0.090135,0.145807,-0.173986,0.191079,0.123180,-0

pd.plotting.scatter_matrix(df, alpha=0.2, figsize=(40,40))

plt.show()

In [6]:
output=df.describe()
#output.to_csv('cold_data')
output

,Percent of GDP based on water intensive industries_2015,Percentage of workforce in Farming Fishing and Forestry 2015,Baseline water stress_2014,Readiness to accept adaptation investment_2015,Tax incentives for renewable energy_2015,Total number of federal public corruption convictions by district_2015,Civic engagement_2010,General innovation capabilities_2015,Estimated percent of adults who think global warming is already harming people in the US now or within 10 years_2014,Percent of population with less than 12th grade education_2015,Population density_2015,Water quality_2015,City debt_2010,Percent of population in floodzone_2015,Percent of buildings in floodzone_2015,Percent of cars in floodzone_2015,Percent of area that is impervious surface_2011,Number of acute care hospital beds available per 1000 residents_2015,Percent of population that is 65 years old or older living alone_2015,Percent of population that is 5 years old or younger_2015,Percent of population with health insurance_2015,Percent of buildings built before 1999_2015,Percent of population residing in the mobile homes_2015,Percent of population spending over 50 percent of income on rent_2015,Percent of land covered by tree canopy_2011,Percent of houses with heating_2015,Percent of population with a disability_2015,Percent of population in poverty_2015,Percent of buildings built before 1979_2015,Outdoor workers_2015,Percent of households without access to a vehicle_2015,Percent of population in 1ft sea level rise zone_2015,Percent of population in 3ft sea level rise zone_2015,Average economic impact of 1ft sea level rise_2015,Average economic impact of 3ft sea level rise_2015,Unnamed: 40,Latitutde,Longitude,City Area_KM,2010 Population,2011 Population,2012 Population,2013 Population,2014 Population,2015 Population,Median Income
count,278.000000,278.000000,277.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,274.000000,278.000000,278.000000,275.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,87.000000,87.000000,87.000000,87.000000,0.0,278.000000,278.000000,278.000000,2.780000e+02,2.780000e+02,2.780000e+02,2.780000e+02,2.780000e+02,2.780000e+02,278.000000
mean,13.426946,0.447039,3.157567,11.618705,1.050360,18.802158,0.628717,0.000569,44.284173,14.802518,1629.363349,3.129496,2099.509873,13.788885,13.926403,15.257810,33.974430,5.158201,9.047665,6.764388,85.298201,84.693741,1.957194,26.522662,17.233983,0.974507,11.735971,16.066832,58.519784,12.017410,10.575180,4.735357,7.428345,0.017125,0.031491,NaN,36.476227,-98.146780,256.347503,2.961474e+05,2.987375e+05,3.015258e+05,3.045125e+05,3.078517e+05,3.111112e+05,52856.928058
std,8.712403,1.387772,1.644446,9.181205,2.334249,18.329330,0.168798,0.000967,6.781412,7.216740,1299.318754,4.599222,1875.825694,14.658425,15.046380,27.795487,14.365784,3.587651,2.351106,1.107600,5.434470,10.914285,1.818160,4.708648,13.419223,0.100084,3.105230,8.193451,20.936214,4.213674,7.685975,7.190017,9.708439,0.040747,0.066497,NaN,5.667576,17.744922,407.807081,5.878017e+05,5.911483e+05,5.958111e+05,6.007524e+05,6.069727e+05,6.128256e+05,16304.416709
min,0.000000,0.020128,0.000000,0.000000,0.000000,0.000000,0.074595,0.000000,0.000000,2.400000,56.592353,0.000000,17.610000,0.000000,0.000000,0.000000,0.000000,0.000000,2.443208,3.200000,65.400000,31.590000,0.000000,16.100000,0.529454,0.199000,5.500000,2.787969,4.000000,2.110000,1.200000,0.000000,0.011879,0.000003,0.000003,NaN,18.011100,-157.858300,18.266896,4.193500e+04,4.219200e+04,4.225900e+04,4.233100e+04,4.234200e+04,4.257000e+04,16423.000000
25%,9.133366,0.124629,2.216669,0.000000,0.000000,4.000000,0.515853,0.000117,41.250000,9.500000,869.230114,0.000000,998.745000,5.497500,5.457500,4.568092,24.582223,2.504193,7.363559,6.100000,82.600000,81.042500,0.500000,23.425000,6.609895,0.989000,9.500000,10.380349,44.225000,9.452

In [7]:
df.columns

Index(['City', 'State', 'Geo.ID',
       'Percent of GDP based on water intensive industries_2015',
       'Percentage of workforce in Farming Fishing and Forestry 2015',
       'Existence of drought management plans_2015',
       'Existence of water management plan_2015', 'Baseline water stress_2014',
       'Readiness to accept adaptation investment_2015',
       'Tax incentives for renewable energy_2015',
       'Total number of federal public corruption convictions by district_2015',
       'Civic engagement_2010', 'General innovation capabilities_2015',
       'Estimated percent of adults who think global warming is already harming people in the US now or within 10 years_2014',
       'Percent of population with less than 12th grade education_2015',
       'Population density_2015', 'Water quality_2015', 'City debt_2010',
       'Percent of population in floodzone_2015',
       'Percent of buildings in floodzone_2015',
       'Percent of cars in floodzone_2015',
       'Percent of

In [8]:
a=1
df.iloc[:,a].describe()
#charles=(df.iloc[:,a])
#print(df.iloc[20:150,a])
#print(df.iloc[24])


count     278
unique     52
top        CA
freq       66
Name: State, dtype: object

In [9]:
df.iloc[:,a].unique()


array(['TX', 'OH', 'NM', 'VA', 'PA', 'CA', 'AK', 'MI', 'CO', 'GA', 'IL',
       'MD', 'LA', 'PR', 'WA', 'MT', 'AL', 'ID', 'MA', 'CT', 'NY', 'VT',
       'FL', 'IA', 'AZ', 'SC', 'WV', 'NC', 'TN', 'WY', 'MO', 'NJ', 'OR',
       'IN', 'ND', 'WI', 'NV', 'HI', 'MS', 'KS', 'KY', 'NE', 'AR', 'NH',
       'MN', 'OK', 'ME', 'RI', 'UT', 'SD', 'DC', 'DE'], dtype=object)

df1 = df[df.isna().any(axis=1)]
print(df1)

In [10]:

#print(df['flood_hist_cost'].isna())
#df.fillna(0, inplace = True)
row=0
count=0
for value in df.iloc[:,a]:
    if value == "PR":
        print(df.iloc[row,0:2],df.iloc[row,a])
        
        count+=1
    row+=1
print(count)
19, 35, 193, 220

City     Bayam<U+0086>n
State                PR
Name: 19, dtype: object PR
City     Carolina
State          PR
Name: 35, dtype: object PR
City     Ponce
State       PR
Name: 193, dtype: object PR
City     San Juan
State          PR
Name: 220, dtype: object PR
4


(19, 35, 193, 220)

In [11]:
city_indicators=pd.read_csv('data/City Indicators.csv')

# Puerto Rico is being dropped at rows [19, 35, 193, 220] because of data inconsistency
city_indicators.drop([19, 35, 193, 220], inplace = True)
#two cities had their order of magnitude incorrect for % of people in floodzone. Their neighboring values match the lower magnitude'
city_indicators=city_indicators.replace(412.9208681,41.29208681)
city_indicators=city_indicators.replace(101.6959823,10.16959823)

city_indicators_old=df.columns
new_names=['city','state','geo_id','GDP_water_all','perc_fff','d_plan','w_plan','base_w_stress','invest','tax_break','corruption','perc_vote','innovate','climate_real','hs_ed','pop_dens','w_quality','debt','flood_pop','flood_build','flood_car','impervious','beds_1000','alone_65','child_5','health_ins','older_1999','mobile_home','rent_50plus','trees','heating','disabled','poverty','older_1979','work_outside','no_car','pop_under_1ft','pop_under_3ft','impact_plus1ft','impact_plus3ft', 'col_empty','lat','long','county','region','city_km2','pop_2010','pop_2011','pop_2012','pop_2013','pop_2014','pop_2015','median_income']
indicators_dict = dict(zip(new_names, city_indicators_old))
city_indicators.columns = new_names
city_indicators.drop(columns =['col_empty'], inplace= True)

feature_names['city_indicators']=indicators_dict

city_indicators.head()



,city,state,geo_id,GDP_water_all,perc_fff,d_plan,w_plan,base_w_stress,invest,tax_break,corruption,perc_vote,innovate,climate_real,hs_ed,pop_dens,w_quality,debt,flood_pop,flood_build,flood_car,impervious,beds_1000,alone_65,child_5,health_ins,older_1999,mobile_home,rent_50plus,trees,heating,disabled,poverty,older_1979,work_outside,no_car,pop_under_1ft,pop_under_3ft,impact_plus1ft,impact_plus3ft,lat,long,county,region,city_km2,pop_2010,pop_2011,pop_2012,pop_2013,pop_2014,pop_2015,median_income
0,Abilene,TX,1600000US4801000,17.148912,0.243117,Yes,No,3.368071,19,0,48,0.496519,0.000052,39,16.5,419.729649,8,759.37,31.28,35.77,37.557516,16.406787,6.537071,10.246056,7.3,82.4,88.57,3.3,23.5,15.915399,0.998,15.7,14.378483,63.4,12.96,5.6,NaN,NaN,NaN,NaN,32.4487,-99.7331,Taylor,South,291.201730,116637,117566,118484,119721,120776,122226,43189
1,Akron,OH,1600000US3901000,16.758186,0.192792,No,No,3.376585,18,1,18,0.722946,0.000587,40,13.7,1239.715136,2,3022.84,7.06,6.91,8.047370,36.647418,10.245602,11.555375,6.7,86.7,95.61,0.3,28.2,26.517394,0.996,15.2,27.106735,85.7,11.74,15.2,NaN,NaN,NaN,NaN,41.0814,-81.5190,Summit,Ohio Valley,159.979494,202814,201111,199955,199038,198492,198329,34512
2,Albuquerque,NM,1600000US3502000,5.659316,0.237722,Yes,Yes,5.000000,20,3,12,0.643955,0.000196,48,11.0,1139.837021,1,1660.55,5.44,5.35,5.295091,26.860472,3.172137,10.636296,6.6,86.0,81.37,3.8,26.2,4.861576,0.998,12.9,15.408964,50.6,10.21,7.4,NaN,NaN,NaN,NaN,35.0853,-106.6056,Bernalillo,Southwest,487.869748,531403,539000,545083,549812,553576,556092,47030
3,Alexandria,VA,1600000US5101000,2.402035,0.066973,No,Yes,2.645670,21,1,40,0.686521,0.000181,51,8.5,3819.963313,0,2217.47,11.09,12.38,11.347841,44.933520,2.129726,7.624217,7.5,85.7,87.71,0.1,17.5,21.723042,0.995,6.2,4.868494,66.9,7.58,9.3,6.319526,8.441215,0.051605,0.067121,38.8048,-77.0469,Arlington,Southeast,39.088072,133647,136968,140337,143684,146422,149315,89134
4,Allentown,PA,1600000US4202000,15.050855,0.271557,No,No,3.145631,15,0,27,0.666227,0.000493,42,22.7,2581.110840,3,1165.14,12.23,11.87,29.317803,43.781505,1.802769,9.865677,7.6,83.2,94.72,0.3,32.7,19.014020,0.996,18.5,28.384735,85.1,19.98,19.6,NaN,NaN,NaN,NaN,40.6084,-75.4902,Lehigh,Northeast,46.205300,116398,117386,117942,118285,118793,119261,36930


# Cleaning City Indicators

In [12]:
import numpy as np


count =0
cleaned = city_indicators.copy()
cleaned.replace('Bayam<U+0086>n','Bayamon')
cities=cleaned['city']
cleaned.drop(columns=['city','state','geo_id','county', 'region','pop_2010', 'pop_2011', 'pop_2012', 'pop_2013', 'pop_2014'], inplace=True)
features=cleaned.columns
cleaned['d_plan'] = cleaned['d_plan'].map({'Yes': 1, 'No': 0})
cleaned['w_plan'] = cleaned['w_plan'].map({'Yes': 1, 'No': 0})
cleaned.fillna(0, inplace=True)


# Calculate summary values for each hazard

In [13]:

# frame one (calc_from_raw) creates the six values for each hazard
calc_from_raw =pd.DataFrame()
calc_from_raw['cities']= cities


calc_from_raw['econ_c']=cleaned['debt']+cleaned['invest']+cleaned['tax_break']
calc_from_raw['gov_c']=cleaned['innovate']+cleaned['climate_real']
calc_from_raw['social_c']=cleaned['corruption']+cleaned['perc_vote']
#make sure to use flood_expos for flood. Use this one for the rest
calc_from_raw ['expos_c']=cleaned['pop_2015']

#heat - adaption and sensitivity formulas unique
calc_from_raw['heat_adapt_c']=cleaned['beds_1000']+cleaned['trees']
calc_from_raw['heat_sens_c']=cleaned['alone_65']+cleaned['poverty']+cleaned['disabled']



#cold - adaption and sensitivity formulas unique
calc_from_raw['cold_adapt_c']=cleaned['beds_1000']+cleaned['heating']
calc_from_raw['cold_sens_c']=cleaned['alone_65']+cleaned['work_outside']+cleaned['child_5']

#flood - exposure, sensitivity and adaption formulas unique
calc_from_raw['flood_expos_c']=cleaned['flood_pop']+cleaned['flood_build']
calc_from_raw['flood_sens_c']=cleaned['older_1999']+cleaned['mobile_home']
calc_from_raw['flood_adapt_c']=cleaned['beds_1000']+cleaned['w_quality']


#drought - sensitivity and adaption unique
calc_from_raw['drought_sens_c']=cleaned['GDP_water_all']+cleaned['perc_fff']
calc_from_raw['drought_adapt_c']=cleaned['d_plan']+cleaned['w_plan']
calc_from_raw.dropna(inplace=True)

# Calculate Summary Values for categories

In [14]:
''' Study standardized then used standardized versions
next stage
'''
from sklearn.preprocessing import StandardScaler

prescale_table=calc_from_raw.drop(columns=['cities'])
prescale_features=prescale_table.columns
scaler=StandardScaler()
calc_hazard_stand=scaler.fit_transform(prescale_table)
calc_from_raw_scaled =pd.DataFrame(calc_hazard_stand, columns=prescale_features)
#calc_from_raw_scaled['cities']= cities


risk_recovery_calc=pd.DataFrame()
risk_recovery_calc['cities']= cities
#Readiness is the value for all hazards so it will only be done once
risk_recovery_calc['readiness_c']=(calc_from_raw_scaled['gov_c']+calc_from_raw_scaled['social_c']+calc_from_raw_scaled['econ_c'])/3

                                                                                                         
risk_recovery_calc['heat_risk']=(calc_from_raw_scaled['expos_c']+calc_from_raw_scaled['heat_adapt_c']+(1-calc_from_raw_scaled['heat_sens_c']))/3
risk_recovery_calc['cold_risk']=(calc_from_raw_scaled['expos_c']+calc_from_raw_scaled['cold_adapt_c']+(1-calc_from_raw_scaled['cold_sens_c']))/3
risk_recovery_calc['flood_risk']=(calc_from_raw_scaled['flood_expos_c']+calc_from_raw_scaled['flood_adapt_c']+(1-calc_from_raw_scaled['flood_sens_c']))/3
risk_recovery_calc['drought_risk']=(calc_from_raw_scaled['expos_c']+calc_from_raw_scaled['drought_adapt_c']+(1-calc_from_raw_scaled['drought_sens_c']))/3
risk_recovery_calc['risk_c']=risk_recovery_calc['heat_risk']+risk_recovery_calc['cold_risk']+risk_recovery_calc['flood_risk']     
risk_recovery_calc.dropna(inplace=True)

fig = plt.figure()

count=1
drop_list=list(city_indicators.columns)
for header in city_indicators.columns:
    x=city_indicators[header]
    for header2 in drop_list:
        plt.show(count)
        y=city_indicators[header2]
        ax = fig.add_subplot()
        plt.scatter(x, y, alpha=0.5)
        
        drop_list.remove(header2)
        count+=1

In [15]:
flood_data=pd.read_csv('data/Flood Data.csv')
flood_data_old=df.columns
new_names=['city', 'state','geo_id','flood_risk','flood_vuln','flood_expose','flood_sens','flood_adapt','flood_ready','flood_social','flood_gov','flood_econ','flood_hist_cost','flood_future_cost','flood_prob_2040','flood_prob_2019_40']
flood_data.columns = new_names


flood_dict = dict(zip(new_names, flood_data_old))
feature_names['flood']=flood_dict

In [16]:
cold_data=pd.read_csv('data/Cold Data.csv')
cold_data_old=df.columns
new_names=['city', 'state', 'geo_id', 'cold_risk', 'cold_vuln', 'cold_expose','cold_sens', 'cold_adapt', 'cold_ready', 'cold_social', 'cold_govG','cold_econ', 'cold_hist_cost', 'cold_p_heat2040']
cold_data.columns = new_names


cold_dict = dict(zip(new_names, cold_data_old))
feature_names['cold_data']=cold_dict
#cold_data.head()

In [17]:
risk_recovery=pd.read_csv('data/Overall Risk & Readiness Scores.csv')
risk_recovery_old=risk_recovery.columns
new_names=['city', 'state', 'geo_id','overall_risk', 'overall_readiness']
risk_recovery.columns = new_names


risk_recovery_dict = dict(zip(new_names, risk_recovery_old))
feature_names['risk_recovery']=risk_recovery_dict
risk_recovery.columns

Index(['city', 'state', 'geo_id', 'overall_risk', 'overall_readiness'], dtype='object')

# Classification based on published Data:

Risk High/low set at 38.40
Readiness high/low set at 46.51

In [50]:
risk_published=risk_recovery.copy()
# Puerto Rico is being dropped at rows [19, 35, 193, 220] because of data inconsistency
risk_published.drop([19, 35, 193, 220], inplace = True)
risk_published['overall_risk'] = np.where(risk_published['overall_risk']>=.3840, 1, 0)
risk_published['overall_readiness'] = np.where(risk_published['overall_readiness']>=.4651, 1, 0)

In [51]:


risk_published

,city,state,geo_id,overall_risk,overall_readiness
0,Abilene,TX,1600000US4801000,1,0
1,Akron,OH,1600000US3901000,1,1
2,Albuquerque,NM,1600000US3502000,0,1
3,Alexandria,VA,1600000US5101000,1,1
4,Allentown,PA,1600000US4202000,1,0
...,...,...,...,...,...
273,Wilmington (DE),DE,1600000US1077580,1,1
274,Wilmington (NC),NC,1600000US3774440,1,1
275,Winston-Salem,NC,1600000US3775000,0,1
276,Worcester,MA,1600000US2582000,1,0


In [19]:
drought_data=pd.read_csv('data/Drought Data.csv')
drought_data.drop(columns=['Unnamed: 14'], inplace=True)
old_columns=drought_data.columns
new_names=['city', 'state', 'geo_id','drought_risk', 'drought_vuln', 'drought_expose','drought_sens', 'drought_adapt', 'drought_ready', 'drought_social', 'drought_gov','drought_econ', 'drought_hist_cost','drought_future_cost','drt_dev_forecast','drt_dev_for_vs_norm','drt_.5sdev_G','drt_.5sdev_G','drt_dist_sdev']
drought_data.columns = new_names


drought_dict = dict(zip(new_names, old_columns))
feature_names['drought_data']=drought_dict


In [20]:
heat_data=pd.read_csv('data/Heat Data.csv')
new_names=['city','state','geo_id','heat_risk','heat_vuln','heat_expose','heat_sens','heat_adapt','heat_ready','heat_social','heat_gov','heat_econ','heat_hist_cost','heat_prob_2040']
old_columns=heat_data.columns
heat_data.columns = new_names

heat_data_dict = dict(zip(new_names, old_columns))
feature_names['heat_data']=heat_data_dict
heat_data.columns

Index(['city', 'state', 'geo_id', 'heat_risk', 'heat_vuln', 'heat_expose',
       'heat_sens', 'heat_adapt', 'heat_ready', 'heat_social', 'heat_gov',
       'heat_econ', 'heat_hist_cost', 'heat_prob_2040'],
      dtype='object')

In [21]:
historical=pd.read_csv('data/Historical Hazards Outcome Data.csv')
new_names=['city', 'state', 'geo_id','fl_instances', 'fl_inj_direct' 
           ,'fl_inj_indirect','fl_inj_total'\
           , 'fl_death_direct' ,'fl_death_indirect','fl_death_total',
           'flood_damage'\
           ,'ht_instances', 'ht_inj_direct' ,'ht_inj_indirect','ht_inj_total'\
           , 'ht_death_direct' ,'ht_death_indirect','ht_death_total','heat_damage'\
          ,'cld_instances', 'cld_inj_direct' ,'cld_inj_indirect','cld_inj_total'\
           ,'cld_death_direct' ,'cld_death_indirect','cld_death_total','cold_damage'\
           ,'coast_fl_instances', 'coast_fl_inj_direct' ,'coast_fl_inj_indirect','coast_fl_inj_total'\
           , 'coast_fl_death_direct' ,'coast_fl_death_indirect','coast_fl_death_total','coast_fl_damage']
 
old_columns=historical.columns
historical.columns = new_names
historical_dict = dict(zip(new_names, old_columns))
feature_names['historical']=historical_dict
#historical.columns

In [22]:
sea_level_data=pd.read_csv('data/Sea Level Rise Data.csv')
new_names=['city', 'state', 'geo_id', 'sea_rise_risk', 'sea_rise_vuln', 'sea_rise_expos',
       'sea_rise_sens', 'sea_rise_adapt', 'sea_rise_ready', 'sea_rise_social', 'sea_rise_gov',
       'sea_rise_econ', 'sea_rise2040', 'sea_rise_cost+1ft']

old_columns=sea_level_data.columns
sea_level_data.columns = new_names
sea_level_data_dict = dict(zip(new_names, old_columns))
feature_names['sea_level_data']=sea_level_data_dict
sea_level_data.columns

Index(['city', 'state', 'geo_id', 'sea_rise_risk', 'sea_rise_vuln',
       'sea_rise_expos', 'sea_rise_sens', 'sea_rise_adapt', 'sea_rise_ready',
       'sea_rise_social', 'sea_rise_gov', 'sea_rise_econ', 'sea_rise2040',
       'sea_rise_cost+1ft'],
      dtype='object')

# Cleaning and Scaling of Raw Values in City Indicators

In [23]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
X=scaler.fit_transform(cleaned)
X.shape
print(X)

[[ 0.41208037 -0.1475303   1.67497927 ...  0.07965868 -0.30922554
  -0.63305173]
 [ 0.3670083  -0.18360462 -0.59702231 ... -0.24071023 -0.18566823
  -1.17645891]
 [-0.91329825 -0.15139734  1.67497927 ...  0.55980838  0.39517923
  -0.39250465]
 ...
 [ 1.16134034 -0.08709063 -0.59702231 ...  0.20712775 -0.12346502
  -0.84015644]
 [ 0.65330453 -0.20677937 -0.59702231 ... -0.39001677 -0.20993549
  -0.49007621]
 [-1.39593872 -0.27012992 -0.59702231 ... -0.51588225 -0.18387258
   0.36019915]]


In [24]:
# Creating derived variables 

# kmeans model  on scaled city indicators

In [25]:
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import pairwise_distances

'''
for value in range(2,100):
'''
kmeans = KMeans(n_clusters=4, random_state=9)
kmeans.fit(X)
kmeans.transform
labels=kmeans.labels_
print(value, metrics.silhouette_score(X, labels, metric='euclidean'))

#silhouette never goes over .11, varies between .07-10 for all 98 cluster sizes

cluster_map = pd.DataFrame()
cluster_map['data_index'] = cities
cluster_map['cluster'] = kmeans.labels_


NY 0.08787935815873825


In [26]:
from collections import defaultdict
row=0
count=0
clust_dict = defaultdict(list)
for value in cluster_map['cluster']:
    clust_dict[value].append(cluster_map.iloc[row,0])
    row +=1

for key in clust_dict:
        clust_dict[key].insert(0,f'Cluster {key}')
        print(clust_dict[key])
        
        

['Cluster 3', 'Abilene', 'Akron', 'Allentown', 'Ann Arbor', 'Atlanta', 'Baltimore', 'Baton Rouge', 'Birmingham', 'Bridgeport', 'Buffalo', 'Burlington', 'Cedar Rapids', 'Charleston (WV)', 'Chattanooga', 'Cincinnati', 'Clarksville', 'Cleveland', 'Columbia (SC)', 'Columbus (GA)', 'Columbus (OH)', 'Davenport', 'Dayton', 'Des Moines', 'Detroit', 'Durham', 'Eugene', 'Evansville', 'Everett', 'Fayetteville', 'Fort Wayne', 'Gainesville', 'Grand Rapids', 'Green Bay', 'Greensboro', 'Gresham', 'Hartford', 'High Point', 'Huntsville', 'Independence', 'Indianapolis', 'Jackson', 'Kansas City (KS)', 'Kansas City (MO)', 'Knoxville', 'Lafayette', 'Lansing', 'Lincoln', 'Little Rock', 'Lowell', 'Manchester', 'Memphis', 'Milwaukee', 'Minneapolis', 'Mobile', 'Montgomery', 'Newport News', 'Norman', 'Oklahoma City', 'Omaha', 'Peoria (IL)', 'Pittsburgh', 'Portland (ME)', 'Portland (OR)', 'Providence', 'Pueblo', 'Richmond (VA)', 'Rochester (NY)', 'Rockford', 'Salem', 'Salt Lake City', 'Shreveport', 'South Bend',

In [27]:
centroids=kmeans.cluster_centers_
kmeans_centroids = pd.DataFrame(centroids, columns = features)

kmeans_centroids

,GDP_water_all,perc_fff,d_plan,w_plan,base_w_stress,invest,tax_break,corruption,perc_vote,innovate,climate_real,hs_ed,pop_dens,w_quality,debt,flood_pop,flood_build,flood_car,impervious,beds_1000,alone_65,child_5,health_ins,older_1999,mobile_home,rent_50plus,trees,heating,disabled,poverty,older_1979,work_outside,no_car,pop_under_1ft,pop_under_3ft,impact_plus1ft,impact_plus3ft,lat,long,city_km2,pop_2015,median_income
0,0.130515,0.045318,0.080592,0.162735,0.151384,-0.240010,-0.094543,0.810179,-0.801003,-0.251379,0.281234,1.285753,0.221019,0.111692,-0.336208,-0.247684,-0.244832,-0.363240,0.298893,-0.400400,-0.814461,0.889298,-1.125609,-0.073514,0.571865,0.274125,-0.647967,-0.331325,-0.386715,0.119954,-0.199394,1.104183,-0.328167,-0.269208,-0.295350,-0.179582,-0.204815,-0.514419,-0.572402,0.055272,0.142289,-0.192169
1,-0.094706,-0.001449,0.160312,0.228348,0.381982,-0.465760,-0.126070,-0.024440,0.191797,0.459123,0.000090,-0.668595,-0.147451,-0.055144,-0.241177,-0.146636,-0.148066,-0.171802,-0.136347,-0.502971,-0.402875,-0.318269,0.490708,-0.512221,0.118346,-0.359639,-0.452281,0.104760,-0.665555,-0.872886,-0.632910,-0.597733,-0.587640,-0.192972,-0.227417,-0.185942,-0.198956,0.019211,-0.587126,-0.035039,-0.151633,0.894172
2,-0.605934,-0.146382,-0.196081,-0.144538,-0.168211,0.357143,0.817781,0.205780,-0.087909,-0.221856,0.826434,-0.123198,0.825819,0.318603,0.621403,1.115424,1.108336,0.680960,0.191318,0.170592,0.695078,-0.608898,-0.087016,0.171879,-0.483050,0.147645,0.387842,-0.397225,0.080937,0.207935,0.341771,-0.146189,1.039995,1.533001,1.745896,1.278057,1.389576,-0.543369,0.814877,0.103433,0.529121,-0.091041
3,0.222936,0.026249,-0.123689,-0.252373,-0.376481,0.438112,-0.119245,-0.531776,0.332914,-0.188249,-0.459760,-0.113869,-0.290080,-0.129181,0.198110,-0.115095,-0.112982,0.130196,-0.121662,0.633137,0.602518,-0.023937,0.254445,0.446975,-0.275716,0.110870,0.657249,0.242469,0.804105,0.646187,0.570921,-0.062138,0.359067,-0.208215,-0.236878,-0.177508,-0.190228,0.480469,0.583344,-0.037696,-0.134463,-0.663999


# PCA on Scaled city indicators

In [28]:
import numpy as np
from sklearn.decomposition import PCA

pca = PCA(n_components=4)
pca.fit_transform(X) 


pca_cov=pca.get_covariance
#pca_v=pd.DataFrame(pca_cov)
print(pca.explained_variance_ratio_)  

print(pca.singular_values_)

# these values suck.  Not sure if there is anything in this model 14% for PCA1 is not good
pca.components_

[0.14201467 0.11223901 0.09070118 0.0866396 ]
[40.42653602 35.9394836  32.30772645 31.5760748 ]


array([[ 0.03788109,  0.02613289,  0.05631242,  0.10744505,  0.10546521,
        -0.14113993, -0.09194482,  0.06557577, -0.0142833 ,  0.08493066,
        -0.05042586, -0.11883455, -0.13209076, -0.02831503, -0.15830562,
        -0.07584153, -0.07398028, -0.09429489, -0.11578066, -0.23951395,
        -0.2458878 ,  0.04604695,  0.04675453, -0.24126452,  0.15762282,
        -0.18546499, -0.15984358,  0.00916059, -0.26572113, -0.3132616 ,
        -0.30419432, -0.08858803, -0.32110309, -0.12312773, -0.13553549,
        -0.08058088, -0.09344771, -0.06224688, -0.24294484,  0.00782275,
        -0.08486846,  0.28651377],
       [ 0.01390854, -0.0209475 , -0.04600639, -0.02754608, -0.07276121,
         0.12993461, -0.02417649, -0.21844195,  0.29640874,  0.01117262,
        -0.19219905, -0.37631745, -0.24953414, -0.02279886,  0.03395091,
         0.08728504,  0.08785176,  0.08583656, -0.25483923,  0.12507669,
         0.16878999, -0.19539205,  0.25594791, -0.09877607, -0.0832544 ,
        -0.17042

# Mean Shift clustering on Scaled city indicators
### sklearn's modeling choice page suggested this for the datam

In [29]:
from sklearn.cluster import MeanShift
MeanShift(bandwidth=None, seeds=None, bin_seeding=False, min_bin_freq=1, cluster_all=True, n_jobs=None)

#for band in range (8,20):
band=12
meanshift = MeanShift(bandwidth=band, min_bin_freq=1, n_jobs=-1)
meanshift.fit_predict(X)
#print(meanshift.labels_)
labels=meanshift.labels_
silhouette =metrics.silhouette_score(X, labels, metric='euclidean')
print(f'band={band},sil={silhouette}')

mean_shift_map = pd.DataFrame()
mean_shift_map['data_index'] = cities
mean_shift_map['cluster'] = labels
row=0

mean_shift_dict = defaultdict(list)
for value in mean_shift_map['cluster']:
    mean_shift_dict[value].append(mean_shift_map.iloc[row,0])
    row +=1

for key in mean_shift_dict:
        mean_shift_dict[key].insert(0,f'Cluster {key}')
        print(mean_shift_dict[key])
# default of allowing any cluster size gives 273 clusters (278 rows). silhouette score .02
# have to default bandwidth from 2 - 9 to get <10 clusters 
#allowing outliers (don't have to join a cluster) only knocked out two rows, still 273.
# 2 binsize varied between 2-10 with little effect
#ran a for loop with 120 iterations -  bin has no effect.  Dramatic increase in sil @ 7
#band=7, bin =2,sil=0.18620192496183094
#band=12,sil=0.4501176330012623
#recleaned and re-ran band=13,sil=0.49085931561724966

band=12,sil=0.44230458188340166
['Cluster 0', 'Abilene', 'Akron', 'Albuquerque', 'Alexandria', 'Allentown', 'Amarillo', 'Anaheim', 'Ann Arbor', 'Antioch', 'Arlington', 'Arvada', 'Atlanta', 'Aurora (CO)', 'Aurora (IL)', 'Austin', 'Bakersfield', 'Baltimore', 'Baton Rouge', 'Beaumont', 'Bellevue', 'Berkeley', 'Billings', 'Birmingham', 'Boise City', 'Boston', 'Bridgeport', 'Brownsville', 'Buffalo', 'Burbank', 'Burlington', 'Cambridge', 'Cape Coral', 'Carlsbad', 'Carrollton', 'Cedar Rapids', 'Centennial', 'Chandler', 'Charleston (WV)', 'Charlotte', 'Chattanooga', 'Chesapeake', 'Cheyenne', 'Chicago', 'Chula Vista', 'Cincinnati', 'Clarksville', 'Clearwater', 'Cleveland', 'Colorado Springs', 'Columbia (MO)', 'Columbia (SC)', 'Columbus (GA)', 'Columbus (OH)', 'Concord', 'Coral Springs', 'Corona', 'Corpus Christi', 'Costa Mesa', 'Dallas', 'Daly City', 'Davenport', 'Dayton', 'Denton', 'Denver', 'Des Moines', 'Detroit', 'Downey', 'Durham', 'El Monte', 'El Paso', 'Elgin', 'Elizabeth', 'Elk Grove', 

# Cluster plots for risk/readiness calculated

In [30]:
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import pairwise_distances


kmeans = KMeans(n_clusters=4, random_state=9)


readiness = risk_recovery_calc['readiness_c']
risk_readiness_cities=risk_recovery_calc['cities']
risk_readiness_all =risk_recovery_calc[['readiness_c','risk_c']]
heat_risk=risk_recovery_calc[['readiness_c','heat_risk']]
flood_risk=risk_recovery_calc[['readiness_c','flood_risk']]
drought_risk=risk_recovery_calc[['readiness_c','drought_risk']]
cold_risk=risk_recovery_calc[['readiness_c','cold_risk']]

risk_readiness_all.dropna(inplace=True)
risk_readiness_all

C:\Users\mike\Anaconda3\envs\tf-gpu-cuda8\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,readiness_c,risk_c
0,-0.178936,1.287075
1,-0.240481,0.812628
2,0.047918,0.345484
3,0.894682,1.011878
4,-0.246894,-1.285711
...,...,...
269,-0.085260,0.127295
270,-0.628980,1.673887
271,-0.504421,1.514766
272,-0.187410,0.440844


In [31]:
kmeans.fit_transform(risk_readiness_all)
labels=kmeans.labels_
silhouette =metrics.silhouette_score(risk_readiness_all, labels, metric='euclidean')
print(f'all - sil={silhouette}')

kmeans.fit_transform(heat_risk)
labels=kmeans.labels_
silhouette =metrics.silhouette_score(heat_risk, labels, metric='euclidean')
print(f'heat - sil={silhouette}')

kmeans.fit_transform(flood_risk)
labels=kmeans.labels_
silhouette =metrics.silhouette_score(flood_risk, labels, metric='euclidean')
print(f'flood - sil={silhouette}')

kmeans.fit_transform(drought_risk)
labels=kmeans.labels_
silhouette =metrics.silhouette_score(drought_risk, labels, metric='euclidean')
print(f'drought - sil={silhouette}')

kmeans.fit_transform(cold_risk)
labels=kmeans.labels_
silhouette =metrics.silhouette_score(cold_risk, labels, metric='euclidean')
print(f'cold - sil={silhouette}')

all - sil=0.3653866486473061
heat - sil=0.3500559673517458
flood - sil=0.3697713518144946
drought - sil=0.3611683537584258
cold - sil=0.32992310116469253


In [40]:
from sklearn.cluster import MeanShift
MeanShift(bandwidth=2, seeds=None, bin_seeding=False, min_bin_freq=1, cluster_all=True, n_jobs=-1)
sub_sets=[risk_readiness_all,heat_risk,cold_risk,flood_risk,drought_risk]
'''
for band in range(2,20):
    for risk in sub_sets:
'''

meanshift.fit_predict(heat_risk)
labels=meanshift.labels_
silhouette =metrics.silhouette_score(heat_risk, labels, metric='euclidean')
print(f'band={band},sil={silhouette}')

mean_shift_map = pd.DataFrame()
mean_shift_map['data_index'] = risk_readiness_cities
mean_shift_map['cluster'] = labels
row=0

mean_shift_dict = defaultdict(list)
for value in mean_shift_map['cluster']:
    mean_shift_dict[value].append(mean_shift_map.iloc[row,0])
    row +=1

for key in mean_shift_dict:
        mean_shift_dict[key].insert(0,f'Cluster {key}')
        print(mean_shift_dict[key])

band=2,sil=0.7819536561342895
['Cluster 0', 'Abilene', 'Akron', 'Albuquerque', 'Alexandria', 'Allentown', 'Amarillo', 'Anaheim', 'Anchorage', 'Ann Arbor', 'Antioch', 'Arlington', 'Arvada', 'Atlanta', 'Aurora (CO)', 'Aurora (IL)', 'Austin', 'Bakersfield', 'Baltimore', 'Baton Rouge', 'Beaumont', 'Bellevue', 'Berkeley', 'Billings', 'Birmingham', 'Boise City', 'Boston', 'Bridgeport', 'Brownsville', 'Buffalo', 'Burbank', 'Burlington', 'Cambridge', 'Cape Coral', 'Carlsbad', 'Carrollton', 'Cedar Rapids', 'Centennial', 'Chandler', 'Charleston (SC)', 'Charleston (WV)', 'Charlotte', 'Chattanooga', 'Chesapeake', 'Cheyenne', 'Chicago', 'Chula Vista', 'Cincinnati', 'Clarksville', 'Clearwater', 'Cleveland', 'Colorado Springs', 'Columbia (MO)', 'Columbia (SC)', 'Columbus (GA)', 'Columbus (OH)', 'Concord', 'Coral Springs', 'Corona', 'Corpus Christi', 'Costa Mesa', 'Dallas', 'Daly City', 'Davenport', 'Dayton', 'Denton', 'Denver', 'Des Moines', 'Detroit', 'Downey', 'Durham', 'El Monte', 'El Paso', 'Elgi